In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print (device_name)
import os
os.environ['COLAB_TPU_ADDR']

In [0]:
! ls -l -a

In [0]:
from google.colab import files
files.upload()

In [0]:
! wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
! tar -xvf ssd_mobilenet_v2_coco_2018_03_29.tar.gz

In [0]:
! git clone https://github.com/tensorflow/models.git

! pip install Cython
! pip install pycocotools
! apt-get install protobuf-compiler
! cd models/research/ && protoc object_detection/protos/*.proto --python_out=.

In [0]:
#%reset -f
#! rm -r model
import sys
sys.path.append("models/research/")
sys.path.append("models/research/slim/")
import os
tpu_name = os.environ.get('COLAB_TPU_ADDR', None)
if tpu_name is not None:
  %run models/research/object_detection/model_tpu_main.py --tpu_name=grpc://$tpu_name --pipeline_config_path=ssd_mobilenet_v2_coco.config --model_dir=model/ --alsologtostderr
else:
  %run models/research/object_detection/model_main.py --pipeline_config_path=ssd_mobilenet_v2_coco.config --model_dir=model/ --alsologtostderr

In [0]:
#! XZ_OPT=-9 tar -cJvf model.tar.xz model
#from google.colab import files
#files.download("model.tar.xz")

In [0]:
%reset -f
#! rm -r export
import sys
sys.path.append("models/research/")
sys.path.append("models/research/slim/")
%run models/research/object_detection/export_inference_graph.py --input_type=image_tensor --pipeline_config_path=ssd_mobilenet_v2_coco.config --trained_checkpoint_prefix=model/model.ckpt-5814 --output_directory=export/

In [0]:
! wget https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/tf_text_graph_ssd.py
! wget https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/tf_text_graph_common.py
%run tf_text_graph_ssd.py --input export/frozen_inference_graph.pb --output config.pbtxt --config ssd_mobilenet_v2_coco.config
! zip -9 -j frozen export/frozen_inference_graph.pb config.pbtxt
from google.colab import files
files.download("frozen.zip")

In [0]:
#! 